In [ ]:
from pathlib import Path
from google.colab import drive

drive.mount('/content/drive')

def folder(f: str):
  fp = "/content/drive/MyDrive/optimization-techniques/" + f
  Path(fp).mkdir(parents = True, exist_ok = True)
  return fp

# Benchmark functions
The complete list of functions available can be found at: https://gitlab.com/luca.baronti/python_benchmark_functions




In [ ]:
!pip3 install benchmark_functions
import benchmark_functions as bf
import numpy as np
from scipy.optimize import minimize, rosen
from matplotlib import pyplot as plt

def graph(f, resolution=50, file = None):
  bounds_lower, bounds_upper = f.suggested_bounds()

  x = np.linspace(bounds_lower[0], bounds_upper[0], resolution)
  y = np.linspace(bounds_lower[1], bounds_upper[1], resolution)
  X, Y = np.meshgrid(x, y)
  Z = np.asarray([[f((X[i][j],Y[i][j])) for j in range(len(X[i]))] for i in range(len(X))])

  fig = plt.figure()
  fig.canvas.set_window_title('Benchmark Function: ' + f._name)
  fig.suptitle(f._name)

  ax = plt.axes(projection='3d')

  ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap='viridis', edgecolor='none')

  ax.view_init(60, 35)
  ax.set_zlabel('z')
  ax.set_xlabel('x')
  ax.set_ylabel('y')

  if file is None:
    plt.show()
  else:
    plt.savefig(file, dpi=400)

In [ ]:
drive = folder("fun/")

for (name, fn) in [
    ("Hypershpere", bf.Hypersphere()), 
    ("DeJong3", bf.DeJong3()), 
    ("PichenyGoldsteinAndPrice", bf.PichenyGoldsteinAndPrice()),
    ("Ackley", bf.Ackley())
    ]:
  graph(fn, file = drive + name + ".svg")